In [1]:
import os
from pyannote.audio import Pipeline
import whisper
import torch

/home/debral/DevSpace/small_transcriptor/.venv/lib/python3.13/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/home/debral/DevSpace/small_transcriptor/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# ============= Настройки =============
file_name = "32140-312ds213-91094-2134.mp3"   # путь к аудио
num_speakers = 4                             # количество спикеров
model_name = "base"                          # whisper: tiny / base / small / medium / large

# Hugging Face токен
hf_auth_token = ""

# ============= Загрузка моделей =============
print("Загрузка моделей...")
model = whisper.load_model(model_name)


Загрузка моделей...


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=hf_auth_token
)

pipeline = pipeline.to(device)

# ============= Диаризация =============
print("Запуск диаризации...")
diarization = pipeline(file_name, num_speakers=num_speakers)

# Собираем сегменты по спикерам
speaker_segments = []
for turn, _, speaker in diarization.itertracks(yield_label=True):
    speaker_segments.append({
        'speaker': speaker,
        'start': turn.start,
        'end': turn.end
    })



Запуск диаризации...


/home/debral/DevSpace/small_transcriptor/.venv/lib/python3.13/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1839.)
  std = sequences.std(dim=-1, correction=1)
/home/debral/DevSpace/small_transcriptor/.venv/lib/python3.13/site-packages/torchaudio/_backend/soundfile_backend.py:71: UserWarning: The MPEG_LAYER_III subtype is unknown to TorchAudio. As a result, the bits_per_sample attribute will be set to 0. If you are seeing this warning, please report by opening an issue on github (after checking for existing/closed ones). You may otherwise ignore this warning.
  warnings.warn(
/home/debral/DevSpace/small_transcriptor/.venv/lib/python3.13/site-packages/torchaudio/_backend/soundfile_backend.py:71: UserWarning: The MPEG_LAYER_III subtype is unknown to TorchAudio. A

KeyboardInterrupt: 

In [6]:

# ============= Транскрибация =============
print("Запуск транскрибации...")
result = model.transcribe(
    file_name,
    language="ru",
    verbose=True,
    word_timestamps=True
)


Запуск транскрибации...
[00:00.000 --> 00:22.100]  Вот в hazdAKE армии.
[00:28.200 --> 00:29.980]  Солф, Перзкам сums, Короли, капусты, pursue нужно завтудиться в нашим
[00:22.260 --> 00:24.340] usterах.
[00:29.980 --> 00:30.740]  Назад начал.
[00:31.060 --> 00:37.260]  Сейчас до этого занимаюсь проектом Кассаема классификации
[00:38.040 --> 00:41.820]  заявывать для GKH и в дальнейшем ответы на вопросы.
[00:42.580 --> 00:46.020]  И сейчас пришли на более перспективные направления
[00:46.020 --> 00:48.780]  это краски, что работает EI.
[00:50.640 --> 00:58.440]  Это же жюри для конференции, типа КМУ, ППС, где по Тейсу,


KeyboardInterrupt: 

In [ ]:

# ============= Сопоставление спикеров =============
print("Сопоставление спикеров...")
for segment in result['segments']:
    segment_start = segment['start']
    segment_end = segment['end']

    possible_speakers = []
    for speaker_seg in speaker_segments:
        overlap_start = max(speaker_seg['start'], segment_start)
        overlap_end = min(speaker_seg['end'], segment_end)
        if overlap_start < overlap_end:
            overlap_duration = overlap_end - overlap_start
            possible_speakers.append({
                'speaker': speaker_seg['speaker'],
                'overlap_duration': overlap_duration
            })

    speaker_label = "Спикер неизвестен"
    if possible_speakers:
        main_speaker = max(possible_speakers, key=lambda x: x['overlap_duration'])
        speaker_label = main_speaker['speaker']
        if len(possible_speakers) > 1:
            speaker_label += " (основной)"

    print(f"[{speaker_label}] [{segment_start:.1f}-{segment_end:.1f}с]: {segment['text']}")